In [1]:
import data_prep
import numpy as np
import pandas as pd

C:\Users\leehk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
FILENAME = "upbit_l2_orderbook_ADA"
PATH = "./data/"
l2_df = pd.read_pickle(PATH+FILENAME)
l2_df[:2]
# TP - trade price TQ - trade Quantity
# BP - bid price, BQ - bid Quantity
# AP - ask price, AQ - ask Quantity

,Time,TP,TQ,BP1,BP2,BP3,BP4,BP5,BP6,BP7,...,BQ5,BQ6,BQ7,AQ1,AQ2,AQ3,AQ4,AQ5,AQ6,AQ7
0,1518934096,435,4184.889,434,433,432,431,430,429,428,...,1148176.290,277861.445,613441.514,53516.415,259857.487,252548.656,186466.125,238298.706,486509.147,440587.075
1,1518934096,435,4184.889,434,433,432,431,430,429,428,...,1148176.290,277861.445,613441.514,53721.209,259857.487,252548.656,186466.125,238298.706,486509.147,440587.075


<img src="./features.png" width="80%">

In [3]:
# make features from dataset
# input shape (batch_size, window_size, n_features)
# ouput shape (batch_size, n_classes)

# if trade price[current+5] = trade price[current] ... 0
#    trade price[current+5] < trade price[current] ... 1
#    trade price[current+5] > trade price[current] ... 2
data = data_prep.get_test_data(window_size=10, predict_step=5, filename=FILENAME)

7186
7186
872.0 records with = (same)
857.0 records with < (plus profit) 
5457.0 records with > (loss)
x shape: (7186, 10, 48)
y shape: (7186, 3)


In [4]:
from nn import NeuralNetwork
from rnn import RNN

n_cross_validation = 3

# input_shape <- (timestep, n_features)
# output_shape <- n_classes
nn = NeuralNetwork(RNN(input_shape=data.x.shape[1:], output_dim=data.y.shape[1]), class_weight= {0:1., 1:1., 2:1.})

print("TRAIN")
nn.train(data)

print("TEST")
nn.test(data)

print("TRAIN WITH CROSS-VALIDATION")
nn.run_with_cross_validation(data, n_cross_validation)

Build model...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Compile model...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
TRAIN
Train...
Train on 5389 samples, validate on 1797 samples
Epoch 1/200
5389/5389 [==============================] - 6s 1ms/step - loss: 0.9000 - loss_class_accuracy: 0.7614 - val_loss: 0.7365 - val_loss_class_accuracy: 0.7813
Epoch 2/200
5389/5389 [==============================] - 1s 193us/step - loss: 0.7940 - loss_class_accuracy: 0.7520 - val_loss: 0.7313 - val_loss_class_accuracy: 0.7813
Epoch 3/200
5389/5389 [==============================] - 1s 205us/step - loss: 0.7839 - loss_class_accuracy: 0.7521 - val_loss: 0.7274 - val_loss_class_accuracy: 0.7813
Epoch 4/200
5389/5389 [==============================] - 1s 204us/step - loss: 0.7785 - loss_class_accuracy: 0.7521 - val_loss: 0.7282 - val_loss_class_accuracy: 0.7813
Epoch 5/200


KeyboardInterrupt: 